In [2]:
import os
os.getcwd()

'/Users/kjh/Documents/innocity'

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

BASE = Path('/Users/kjh/Documents/innocity/data')  # 네 로컬 경로로 바꿔도 됨

file_recent = BASE / "행정구역_읍면동_별_5세별_주민등록인구_2011년__20251124180210.csv"
file_early  = BASE / "행정구역_읍면동_별_5세별_주민등록인구_20251124180650.csv"

# 1) 파일 읽기 ---------------------------------------------------------
df1 = pd.read_csv(file_recent, encoding="cp949")
df2 = pd.read_csv(file_early,  encoding="cp949")

# 2) 최근 파일(df1) 정리: 헤더 행 제거 + 코드/이름 추출 ---------------
df1 = df1[df1["13999001 항목"] != "13999001 항목"].copy()

df1["code"] = df1["A 행정구역(동읍면)별"].str.extract(r"^(\d+)", expand=False)
df1["name"] = df1["A 행정구역(동읍면)별"].str.replace(r"^\d+\s*", "", regex=True)

# 10자리 코드(읍면동) + 경상북도(2자리)만 있음 → 읍면동만 사용
df1 = df1[df1["code"].str.len() == 10].copy()

# 3) 과거 파일(df2) 정리: 헤더 행 제거 + T2 인구만 -------------------
df2 = df2[df2["13999001 항목"] != "13999001 항목"].copy()
df2 = df2[df2["13999001 항목"] == "T2 인구"].copy()

admin_cols = [c for c in df2.columns if c.startswith("A 행정구역(동읍면)별")]

def extract_code_and_name(row):
    for col in admin_cols[::-1]:  # 가장 깊은 수준부터
        s = str(row[col])
        m = re.match(r"\s*(\d{10})\s*(.*)", s)
        if m:
            return pd.Series({"code": m.group(1), "name": m.group(2)})
    return pd.Series({"code": None, "name": None})

df2_extra = df2.apply(extract_code_and_name, axis=1)
df2 = pd.concat([df2.reset_index(drop=True), df2_extra], axis=1)
df2 = df2[~df2["code"].isna()]  # 시도/시군구 소계 제거

def parse_pop(val):
    """'-', '0 계' 등은 NA, 나머지는 숫자로"""
    if isinstance(val, str):
        v = val.strip()
        if v in ["", "-", "0 계", "0계"]:
            return np.nan
        v = v.replace(",", "")
        try:
            return float(v)
        except:
            return np.nan
    try:
        return float(val)
    except:
        return np.nan

# df1: 2011–2024 -------------------------------------------------------
year_cols1 = [c for c in df1.columns if c.startswith("Y")]

data1_long = df1.melt(
    id_vars=["code", "name"],
    value_vars=year_cols1,
    var_name="year_col",
    value_name="pop_raw"
)
data1_long["year"] = data1_long["year_col"].str.extract(r"Y(\d{4})")[0].astype(int)
data1_long["pop"]  = data1_long["pop_raw"].apply(parse_pop)

# df2: 1997–2010 ------------------------------------------------------
year_cols2 = [c for c in df2.columns if c.startswith("Y")]

data2_long = df2.melt(
    id_vars=["code", "name"],
    value_vars=year_cols2,
    var_name="year_col",
    value_name="pop_raw"
)
data2_long["year"] = data2_long["year_col"].str.extract(r"Y(\d{4})")[0].astype(int)
data2_long["pop"]  = data2_long["pop_raw"].apply(parse_pop)


In [5]:
# 이름→코드 테이블 (참고용)
df1_units = df1[["code","name"]].drop_duplicates()
df2_units = df2[["code","name"]].drop_duplicates()
combined_units = pd.concat([df1_units, df2_units]).drop_duplicates()

# 행정구역 변경 매핑 (old_code -> new_code)
rename_map = {
    # 10) 사벌면 → 11) 사벌국면 (2020.1.1)
    "4725032000": "4725032500",

    # 12) 압량읍(신설) ← 13) 압량면(폐지) (2020.1.1)  *실제 코드는 반대로: old=면, new=읍
    "4729036000": "4729025600",

    # 23) 고령읍 → 24) 대가야읍 (2015.4.2)
    "4783025000": "4783025300",

    # 30) 호명면 → 27) 호명읍 (2024.2.1 승격)
    "4790036000": "4790025300",

    # 33) 서면 → 35) 금강송면 (2015.4.1, 울진군 서면만)
    "4793032000": "4793039000",

    # 34) 원남면 → 36) 매화면 (2015.4.1, 울진군 원남면만)
    "4793034000": "4793040000",

    # 28) 상리면 → 31) 효자면 (2016.2.1)
    "4790032000": "4790042000",

    # 29) 하리면 → 32) 은풍면 (2016.2.1)
    "4790033000": "4790043000",

    # 2) 양북면 → 3) 문무대왕면 (2021.4.1)
    "4713031000": "4713031500",

    # 7–9) 공단1동 + 공단2동 → 공단동 (2021.1.1 통합)
    "4719062100": "4719070000",   # 공단1동
    "4719062200": "4719070000",   # 공단2동

    # 25) 금수면 → 26) 금수강산면 (2024.8.1)
    "4784035000": "4784035500",
}

def canon_code(code: str) -> str:
    code = str(code)
    return rename_map.get(code, code)

data1_long["canon_code"] = data1_long["code"].astype(str).apply(canon_code)
data2_long["canon_code"] = data2_long["code"].astype(str).apply(canon_code)


In [6]:
# 2024년에 관측되는 이름을 canonical name으로 사용
latest_2024 = data1_long[data1_long["year"] == 2024].copy()
latest_2024 = latest_2024[latest_2024["canon_code"].str.len() == 10]

canon_name_map = latest_2024.set_index("canon_code")["name"].to_dict()

def canon_name(row):
    c = row["canon_code"]
    return canon_name_map.get(c, row["name"])

data1_long["canon_name"] = data1_long.apply(canon_name, axis=1)
data2_long["canon_name"] = data2_long.apply(canon_name, axis=1)

# 읍면동(10자리 코드)만 남기기
data1_long = data1_long[data1_long["canon_code"].str.len() == 10].copy()
data2_long = data2_long[data2_long["canon_code"].str.len() == 10].copy()


In [7]:
panel = pd.concat([data2_long, data1_long], ignore_index=True)

# 동일 canonical code·연도에 여러 행이 모일 수 있음(공단1/2동 → 공단동 같은 경우)
# → NA가 하나도 없으면 합계, 전부 NA면 NA
panel_agg = (
    panel.groupby(["canon_code", "canon_name", "year"])["pop"]
    .agg(lambda x: x.sum() if x.notna().any() else np.nan)
    .reset_index()
)

panel_agg["year"].min(), panel_agg["year"].max()
# (1997, 2024)


(np.int64(1997), np.int64(2024))

In [16]:
panel_agg.head()

,canon_code,canon_name,year,pop
0,4711051000,구룡포읍,1997,35121.0
1,4711051000,구룡포읍,1998,NaN
2,4711051000,구룡포읍,1999,NaN
3,4711051000,구룡포읍,2000,NaN
4,4711051000,구룡포읍,2001,NaN


### 2024년 읍면동 리스트

In [10]:
eup_2024 = panel_agg[panel_agg["year"] == 2024].copy()

# 2024년 인구가 실제로 있는 읍면동만 (폐지되어 NA인 행 제외)
eup_2024_valid = eup_2024[eup_2024["pop"].notna()].copy()

# 정렬
eup_2024_valid = eup_2024_valid.sort_values(["canon_code"])

# 예: 상위 몇 개 확인
print(eup_2024_valid.head())

# 필요하면 CSV로 저장
eup_2024_valid.to_csv(BASE / "eupmyeondong_list_2024.csv", index=False, encoding="euc-kr")


     canon_code canon_name  year      pop
573  4711125000       구룡포읍  2024   6629.0
601  4711125300        연일읍  2024  28096.0
629  4711125600        오천읍  2024  57405.0
657  4711131000        대송면  2024   3102.0
685  4711132000        동해면  2024   9234.0


In [12]:
# 2024년에 존재하는 읍면동만 대상으로
codes_2024 = eup_2024_valid["canon_code"].unique()
sub = panel_agg[panel_agg["canon_code"].isin(codes_2024)].copy()

years = sorted(panel_agg["year"].unique())
n_years = len(years)  # 28년

comp = (
    sub.groupby(["canon_code", "canon_name"])["pop"]
    .agg(n_years_nonmissing=lambda x: x.notna().sum())
    .reset_index()
)
comp["complete_1997_2024"] = comp["n_years_nonmissing"] == n_years

complete_units   = comp[comp["complete_1997_2024"]].copy()
incomplete_units = comp[~comp["complete_1997_2024"]].copy()

print("완전 패널 읍면동 개수:", len(complete_units))
print("결측 있는 읍면동 개수:", len(incomplete_units))

# 저장
complete_units.to_csv(BASE / "eupmyeondong_complete_1997_2024.csv", index=False, encoding="euc-kr")
incomplete_units.to_csv(BASE / "eupmyeondong_incomplete_1997_2024.csv", index=False, encoding="euc-kr")


완전 패널 읍면동 개수: 225
결측 있는 읍면동 개수: 92


In [17]:
import pandas as pd
import numpy as np
from pathlib import Path

# 1) 2024년에 존재하는 읍면동 코드들만 대상으로 wide 변환
codes_2024 = panel_agg.loc[panel_agg["year"] == 2024, "canon_code"].unique()

sub = panel_agg[panel_agg["canon_code"].isin(codes_2024)].copy()

# 2) wide 형식으로 pivot (행: 읍면동, 열: 연도)
wide = (
    sub.pivot_table(
        index=["canon_code", "canon_name"],
        columns="year",
        values="pop"
    )
    .reset_index()
)

# 3) 열 이름 정리: year → Y1997 식으로
year_cols = [c for c in wide.columns if isinstance(c, (int, np.integer))]
new_cols = {
    y: f"Y{y}" for y in year_cols
}
wide = wide.rename(columns=new_cols)

# 4) 열 순서 정리
ordered_cols = ["canon_code", "canon_name"] + sorted(
    [c for c in wide.columns if c.startswith("Y")]
)
wide = wide[ordered_cols]

# 5) CSV로 저장 (확인용)
wide.to_csv(BASE / "emd_2024_check.csv", index=False, encoding="euc-kr")

# 6) 상위 몇 행만 미리 보기
wide.head()


year,canon_code,canon_name,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,...,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,Y2022,Y2023,Y2024
0,4711125000,구룡포읍,NaN,26605.0,29938.0,31255.0,32223.0,32363.0,32400.0,32455.0,...,8900.0,8670.0,8400.0,8177.0,7885.0,7568.0,7335.0,7083.0,6827.0,6629.0
1,4711125300,연일읍,NaN,38902.0,39074.0,39369.0,39973.0,41945.0,41970.0,41486.0,...,34246.0,33859.0,33375.0,32478.0,31611.0,30633.0,30278.0,29834.0,29134.0,28096.0
2,4711125600,오천읍,NaN,9154.0,8884.0,8487.0,8040.0,7741.0,7409.0,7155.0,...,53743.0,55954.0,56811.0,56115.0,55622.0,54839.0,56185.0,55864.0,57679.0,57405.0
3,4711131000,대송면,NaN,16235.0,15751.0,15460.0,15076.0,14679.0,14534.0,14217.0,...,4436.0,4207.0,4041.0,3820.0,3638.0,3500.0,3396.0,3326.0,3220.0,3102.0
4,4711132000,동해면,NaN,8005.0,7801.0,7411.0,7050.0,6812.0,6547.0,6390.0,...,10150.0,9876.0,9569.0,9947.0,9967.0,9908.0,9763.0,9695.0,9503.0,9234.0
